In [14]:
pip install --upgrade tavily-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated , List
import operator,os
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from tavily import TavilyClient
from dotenv import load_dotenv
from contextlib import ExitStack

In [2]:
load_dotenv(override = True)
api_key=os.getenv('OPENAI_API_KEY')

In [3]:
from langgraph.checkpoint.sqlite import SqliteSaver

stack = ExitStack()
memory = stack.enter_context(SqliteSaver.from_conn_string(":memory:"))

In [4]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [5]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""

In [6]:
WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

In [7]:
REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

In [8]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""


In [9]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""


In [10]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries : List[str]

C:\Users\Jumanapa\anaconda3\envs\llms\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [12]:
class AgentState(TypedDict):
    task : str
    plan : str
    content : Annotated[List[str] , operator.add]
    draft : str
    critique : str
    revision_number : int
    max_revisions : int

In [13]:
def plan_node(state:AgentState):
    messages = [
        SystemMessage(content = PLAN_PROMPT),
        HumanMessage(content = state['task'])
    ]
    response = model.invoke(messages)
    return {"plan":response}

In [14]:
def research_plan_node(state :AgentState):
    messages = [
        SystemMessage(content = RESEARCH_PLAN_PROMPT),
        HumanMessage(content = state['task'])
    ]
    queries = model.with_structured_output(Queries).invoke(messages)
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q,max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content":content}

In [15]:
def generate_node(state : AgentState):
    content = state['content'] or []
    usermessage = HumanMessage(content = f"Here is my task : {state['task']} \n\n Here is my plan : {state['plan']}")
    messages = [
        SystemMessage(content = WRITER_PROMPT),
        usermessage
    ]
    response = model.invoke(messages)
    return {"draft" : response.content,"revision_number":state.get('revision_number',1)+1}
    

In [16]:
def generate_critique_node(state:AgentState):
    messages = [
        SystemMessage(content = REFLECTION_PROMPT),
        HumanMessage(content = state['draft'])
    ]
    critique = model.invoke(messages)
    return {"critique":critique.content}

In [17]:
def research_critique_node(state:AgentState):
    content = state['content'] or []
    messages =[
        SystemMessage(content = RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content = state['critique'])
    ]
    queries = model.with_structured_output(Queries).invoke(messages)
    for q in queries.queries:
        response = tavily.search(query = q,max_results = 2)
        for r in response['results']:
            content.append(r['content'])
    return {"content":content}

In [18]:
def reach_maximum_revision(state:AgentState):
    return state['revision_number'] > state['max_revisions']

In [19]:
builder = StateGraph(AgentState)

In [20]:
builder.add_node("planning",plan_node)
builder.add_node("research_plan",research_plan_node)
builder.add_node("generate",generate_node)
builder.add_node("reflect",generate_critique_node)
builder.add_node("research_critique",research_critique_node)
builder.add_conditional_edges("generate",reach_maximum_revision,{True : END,False:"reflect"})

In [21]:
builder.add_edge("planning","research_plan")
builder.add_edge("research_plan","generate")
builder.add_edge("reflect","research_critique")
builder.add_edge("research_critique","generate")


In [22]:
builder.set_entry_point("planning")

In [23]:
graph = builder.compile(checkpointer = memory)

In [24]:
thread = {"configurable": {"thread_id": "1"}}
for s in graph.stream({
    'task': "what is the difference between langchain and langsmith",
    "max_revisions": 2,
    "revision_number": 1,
    
}, thread):
    print(s)
    print('--------------------------------')

{'planning': {'plan': AIMessage(content='I. Introduction\n    A. Brief overview of Langchain and Langsmith\n    B. Thesis statement: Exploring the differences between Langchain and Langsmith\n\nII. Langchain\n    A. Definition and purpose\n    B. Key features and characteristics\n    C. Use cases and applications\n    D. Advantages and limitations\n\nIII. Langsmith\n    A. Definition and purpose\n    B. Key features and characteristics\n    C. Use cases and applications\n    D. Advantages and limitations\n\nIV. Comparison between Langchain and Langsmith\n    A. Technology stack\n    B. Scalability and performance\n    C. Security and privacy\n    D. Adoption and popularity\n    E. Future prospects\n\nV. Conclusion\n    A. Recap of key differences between Langchain and Langsmith\n    B. Implications for the future of blockchain technology\n    C. Final thoughts and recommendations\n\nNotes:\n- Ensure to provide clear definitions and examples for both Langchain and Langsmith.\n- Include 